In [ ]:
#import packages required to run scripts
import numpy as np #numpy is numerical python.  It is a package so you can do math
import matplotlib.pyplot as plt #this package allows you to make nice plots
import pandas as pd


#Some basic notes:
# python is a zero-based indexing language.
# Use [] to access elements of an array
# Use () to denote arguments of a function.
# Contrast to matlab:
# If you have used matlab, it is a 1-based indexing language
# In matlab, you use () both to access elements of a matrix and to provide arguments to a function

In [ ]:
filename='Global_Carbon_Budget_2022v1.0.xlsx'
tabname='Historical Budget'
budget=pd.read_excel(filename, sheet_name=tabname, header=15)

budget.rename(columns={"fossil emissions excluding carbonation": "fossil", # get rid of names with spaces
                       "land-use change emissions": "LUC",
                       "atmospheric growth": "atm_growth",
                       "land sink": "land_sink",
                       "ocean sink": "ocean_sink",
                       "cement carbonation sink": "cement_weathering",
                       "budget imbalance": "imbalance"}, inplace=True)
budget #look at the dataframe you have just loaded in


In [ ]:
tabname='Fossil Emissions by Category'
fossil=pd.read_excel(filename, sheet_name=tabname, header=8)
fossil
fossil.rename(columns={"fossil.emissions.excluding.carbonation": "fossil", #get rid of names with spaces
                       "Cement.emission": "cement",
                       "Per.Capita": "percapita"}, inplace=True)
fossil

In [ ]:
# readESRL is a function that reads atmospheric CO2 files
# for a specified site.  It also writes output to a dictionary.

def readESRL(filename):
    with open(filename, 'r') as fid:
        first_line=fid.readline()
        nheader=first_line[-3:-1]
        nheader=int(float(nheader))
    data=np.loadtxt(filename, usecols=(1,2,3), skiprows=nheader)
    time=data[:,0]+data[:,1]/12
    co2=data[:,2]
    month=data[:,1]
    year=data[:,0]
    site['year']=year
    site['month']=month
    site['co2']=co2
    yearlist=np.unique(year)
    Nyears=len(yearlist)
    annCO2=0*yearlist
    for iyr in np.arange(0,Nyears): #since data are monthly, we want to calculate annual averages
        jyr=np.where(year==yearlist[iyr])
        annCO2[iyr]=np.nanmean(co2[jyr])
    site['uniqueyears']=yearlist
    site['annmean_co2']=annCO2
    return site

In [ ]:
#Define dictionary elements for mlo and spo
#point to location where these files are kept on your machine
#read the files for these two sites
#use print statement to "see" the output
#you can comment this out if you don't want to see a bunch of numbers on your screen

mlo={'name':'Mauna Loa', 'acronym': 'mlo'} #initialize a dictionary for mlo
spo={'name':'South Pole', 'acronym': 'spo'}
ESRL=[mlo, spo]
#fileloc='/Users/gkeppela/Dropbox/Courses/F15_CarbonCycle/Fall2019/PS1_CarbonBudget/'
for site in ESRL:
    filename='co2_'+site['acronym']+'_surface-flask_1_ccgg_month.txt'
    site=readESRL(filename)
    #readESRL(fileloc+filename)
#print(mlo) #you can comment this out!  But use print statements to troubleshoot your code as needed

In [ ]:
#Plot a figure showing monthly and annual CO2

plt.figure()
plt.plot(mlo['year']+(mlo['month']-0.5)/12.0, mlo['co2'], color=[1,0,0])
plt.plot(mlo['uniqueyears'], mlo['annmean_co2'], color=[0.7, 0.2, 0.2])
plt.plot(spo['year']+(spo['month']-0.5)/12.0, spo['co2'], color=[0,0,1])
plt.plot(spo['uniqueyears'], spo['annmean_co2'], color=[0.2, 0.2, 0.9])
plt.xlabel('Year')
plt.ylabel(r'CO$_2$ [ppm]')
#plt.savefig('AtmosCO2Timeseries.eps') 

In [ ]:
# Find common period between CO2 and FF data
# The datasets are all different lengths. 
# If you work in Earth science or really any kind of research, this section of code
# represents the types of housekeeping that has to happen when you analyze data from
# different sources.

yrbeg=1981
yrend=2021
print(yrend)
Nyr=yrend-yrbeg+1

mlo_start=np.where(mlo['uniqueyears']==yrbeg)[0]
print(mlo_start)
mlo_end=np.where(mlo['uniqueyears']==yrend)[0]
spo_start=np.where(spo['uniqueyears']==yrbeg)[0]
spo_end=np.where(spo['uniqueyears']==yrend)[0]
ff_start=np.where(fossil.Year==yrbeg)[0]
ff_end=np.where(fossil.Year==yrend)[0]
#print(np.arange(mlo_start,mlo_end))
MLO_common=mlo['annmean_co2'][np.arange(mlo_start,mlo_end+1)]
MLO_month_common=mlo['co2'][np.arange(mlo_start,mlo_end+1)]
SPO_common=spo['annmean_co2'][np.arange(spo_start,spo_end+1)]
FF_common=fossil.Year[np.arange(ff_start,ff_end+1)]
#Year_common=fossil.Year[np.arange(ff_start,ff_end+1)]
Year_common=mlo['uniqueyears'][np.arange(mlo_start,mlo_end+1)]
print(Year_common)

In [ ]:
#Calculate atmospheric growth rate and airborn fraction

ppm2Pg=2 #conversion factor
Catm=0.5*(MLO_common+SPO_common)*ppm2Pg

growthrate=np.zeros(int(Nyr-1))
for iyr in range(len(Year_common)-1):
    print(iyr)
    growthrate[iyr]=Catm[iyr+1]-Catm[iyr]
    
AnnualAF=100*growthrate/FF_common[0:-1]
#find the years that comprise the decade of interest : indx_dec
indx_start=np.where(Year_common==1981)[0]
print(indx_start)
print(Year_common[indx_start])
indx_dec=np.arange(indx_start,indx_start+10)
print(indx_dec)
#np.mean(AnnualAF[indx_dec])
#np.stdev(AnnualAF[index_dec])

plt.figure()
plt.plot(Year_common[0:-1],growthrate)
##What are the units on this figure?

plt.figure()
plt.plot(Year_common[0:-1], AnnualAF)
## What are the units on this figure?

In [ ]:
#Calculate hemispheric gradient in atmospheric CO2 as a function of fossil emissions

HemDelta=MLO_common-SPO_common
plt.plot(FF_common, HemDelta, 'o')
#Add x and y labels, with units


# Fit a slope through this relationship
#Then plot the trend line on the axes.